<a href="https://colab.research.google.com/github/nspiegeln/iml_2022/blob/main/task4_main4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import torch
import numpy as np
from torch import nn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [3]:
#load data
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path = "gdrive/MyDrive/task4/"
df_train= pd.read_csv(path + "train_features.csv")
df_train_labels= pd.read_csv(path + "train_labels.csv")
df_pretrain= pd.read_csv(path +"pretrain_features.csv")
df_pretrain_labels= pd.read_csv(path +"pretrain_labels.csv")
df_test= pd.read_csv(path +"test_features.csv")

In [5]:
df_train_values = df_train.iloc[:,2::]
df_pretrain_values = df_pretrain.iloc[:,2::]

df_train_lab = df_train_labels.iloc[:,1]
df_pretrain_lab = df_pretrain_labels.iloc[:,1]

train = df_train_values.values #dataset with known homo-lumo gap
pretrain = df_pretrain_values.values #dataset with known lumo value

train_labels = df_train_lab.values
pretrain_labels = df_pretrain_lab.values

In [6]:
x_pretrain, x_pretest, y_pretrain, y_pretest = train_test_split(pretrain, pretrain_labels, test_size = 0.1)
print(x_pretrain.shape[0], x_pretrain.shape[1])
print(x_pretest.shape[0], x_pretest.shape[1])

45000 1000
5000 1000


In [92]:
#get device
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

In [7]:
# PRE-PROCESSING of features according to autoencoder
# pre-train set

#hyperparameters
dim = 200
epochs = 100         #resuls still decrease after epoch 100,  @epoch=60 loss=4.7e-3, @epoch=100 loss=3.4e-3
batch_size = 64

# create encoder and decoder to reduce dimensionality of features
class Autoencoder(Model):
  def __init__(self, dim):
    super(Autoencoder, self).__init__()
    self.dim = dim
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(500, activation='relu'),
      layers.Dense(200, activation='relu'),
    ])
    self.trainor = tf.keras.Sequential([
      layers.Dense(100, activation='relu'),
      
    ])
    self.predictor = tf.keras.Sequential([
      layers.Dense(100, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(1),
    ])                                     

  def call(self, x):
    encoded = self.encoder(x)
    trained = self.trainor(encoded)
    predicted = self.predictor(trained)
    return predicted

autoencoder = Autoencoder(dim)
sgd = SGD(learning_rate=0.1, decay=1e-2, momentum=0.9)
autoencoder.compile(optimizer='sgd', loss=losses.MeanSquaredError())

autoencoder.fit(x_pretrain, y_pretrain, epochs=epochs, batch_size = batch_size, shuffle=True, validation_data=(x_pretest, y_pretest))

Epoch 1/100
704/704 [==============================] - 13s 17ms/step - loss: 0.1247 - val_loss: 0.0293
Epoch 2/100
704/704 [==============================] - 9s 13ms/step - loss: 0.0250 - val_loss: 0.0317
Epoch 3/100
704/704 [==============================] - 8s 11ms/step - loss: 0.0199 - val_loss: 0.0229
Epoch 4/100
704/704 [==============================] - 8s 12ms/step - loss: 0.0173 - val_loss: 0.0200
Epoch 5/100
704/704 [==============================] - 10s 14ms/step - loss: 0.0154 - val_loss: 0.0155
Epoch 6/100
704/704 [==============================] - 8s 12ms/step - loss: 0.0141 - val_loss: 0.0145
Epoch 7/100
704/704 [==============================] - 8s 12ms/step - loss: 0.0131 - val_loss: 0.0136
Epoch 8/100
704/704 [==============================] - 8s 11ms/step - loss: 0.0124 - val_loss: 0.0151
Epoch 9/100
704/704 [==============================] - 9s 12ms/step - loss: 0.0117 - val_loss: 0.0124
Epoch 10/100
704/704 [==============================] - 9s 13ms/step - loss: 0.0

In [ ]:
y_predpretrain = autoencoder.predict(x_pretrain)

In [8]:
# Transfer learning to HOMO-LUMO gap learning
# freeze everything but last layer
for layer in autoencoder.layers[:-1]:
  layer.trainable = False

autoencoder.encoder.summary()
autoencoder.trainor.summary()
autoencoder.predictor.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 500)               500500    
                                                                 
 dense_1 (Dense)             (None, 200)               100200    
                                                                 
Total params: 600,700
Trainable params: 0
Non-trainable params: 600,700
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               20100     
                                                                 
Total params: 20,100
Trainab

In [11]:
dim = 1
epochs = 800
class AutoencoderNew(Model):
  def __init__(self, dim):
    super(AutoencoderNew, self).__init__()
    self.dim = dim
    self.encoder = tf.keras.Sequential([
      autoencoder,
      layers.Dense(50, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(1),
    ])                                    

  def call(self, x):
    encoded = self.encoder(x)
    return encoded

model = AutoencoderNew(dim)
sgd = SGD(learning_rate=0.1, decay=1e-2, momentum=0.5)

model.compile(optimizer='sgd', loss=losses.MeanSquaredError())

model.fit(train, train_labels, epochs=epochs, shuffle=True)

Epoch 1/800
4/4 [==============================] - 3s 311ms/step - loss: 3.1659
Epoch 2/800
4/4 [==============================] - 0s 7ms/step - loss: 1.8946
Epoch 3/800
4/4 [==============================] - 0s 6ms/step - loss: 0.9639
Epoch 4/800
4/4 [==============================] - 0s 125ms/step - loss: 0.3529
Epoch 5/800
4/4 [==============================] - 0s 6ms/step - loss: 0.1474
Epoch 6/800
4/4 [==============================] - 0s 6ms/step - loss: 0.0997
Epoch 7/800
4/4 [==============================] - 0s 6ms/step - loss: 0.0916
Epoch 8/800
4/4 [==============================] - 0s 6ms/step - loss: 0.0899
Epoch 9/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0884
Epoch 10/800
4/4 [==============================] - 0s 6ms/step - loss: 0.0863
Epoch 11/800
4/4 [==============================] - 0s 6ms/step - loss: 0.0816
Epoch 12/800
4/4 [==============================] - 0s 6ms/step - loss: 0.0788
Epoch 13/800
4/4 [==============================] - 0s 6m

In [146]:

# # create encoder and decoder to reduce dimensionality of features
# class Homolumo(Model):
#   def __init__(self, dim):
#     super(Homolumo, self).__init__()
#     self.dim = dim   
#     self.inflator = tf.keras.Sequential([
#       layers.Flatten(),
#       layers.Dense(50, activation='relu'),
#       layers.Dense(100, activation='relu'),
#     ])
#     self.deflator = tf.keras.Sequential([
#       layers.Flatten(),
#       layers.Dense(100, activation='relu'),
#       layers.Dense(50, activation='relu'),
#       layers.Dense(dim),
#     ])

#   def call(self, x):
#     inflated = self.inflator(x)
#     deflated = self.deflator(inflated)
#     return deflated

# model = Homolumo(dim)

# sgd = SGD(learning_rate=0.1, decay=1e-2, momentum=0.9)

# model.compile(optimizer='sgd', loss=losses.MeanSquaredError())

# model.fit(train, train_labels, epochs=epochs, shuffle=True) #, validation_data=(y_predicttest, y_pretest))

In [12]:
#predict homolumo gap for testset
x_test = df_test.iloc[:,2::]
id_test = df_test.iloc[:,0]

y_test = model.predict(x_test)

In [13]:
#export solution
df_evaltest = pd.DataFrame(y_test, columns = ['y'])
df_evaltest.insert(0, 'Id', id_test)
df_evaltest.to_csv('solutiontot4.csv', index=False, index_label=False, encoding = 'utf-8')